@nbinclude("Utils.ipynb");

# Computation of policy functions by EGM method

## First helper function

The first function `interpLinear` is a linear interpolation function, that is more specialized and hence slightly faster than the interpolation function of the `LinearInterpolation` package.

The function specification is `interpLinear(x, xs, ys, n, lb)` where:
* `x` is a scalar where the interpolation is computed;
* `xs` and `ys` are sorted vectors such that `(xs[i], ys[i])` for `i=1,...,n` is a set of `n` known points;
* `n` is the common length of vectors `xs` and `ys`;
* `lb` is the lower bound for the truncation of the interpolation. If the interpolation result is lower than lb, the interpolation is trauncated to lb.

The function returns a pair containing:
* a scalar corresponding to the interpolated value (truncated if needed);
* an index `np` which is such that the interpolation is performed between indices `np` and `np+1`.

In [1]:
function interpLinear(x::T, 
                      xs::Vector{T},
                      ys::Vector{T}, 
                      n::I, lb_y::T)::Tuple{T,I} where{T<:Real,I<:Integer}
   
    nx = searchsortedlast(xs, x) 
    #np is the largest index such that xs[np]≤x (and hence xs[np+1]>x). Returns 0 if x≤xs[1]. xs sorted.
        
    #Adjust nx if x falls out of bounds of xs
    if nx == 0
        nx += 1
    elseif (nx==n)
        nx -= 1
    end    

    # Linear interpolation in x between xs[nx] and xs[nx+1]
    x_l,x_h = xs[nx],xs[nx+1]
    y_l,y_h = ys[nx],ys[nx+1]
    y = y_l + (y_h-y_l)/(x_h-x_l)*(x-x_l) 
    
    # returns interpolated value and corresponding index
    return ((y<lb_y) ? lb_y : y), nx
    
end

interpLinear (generic function with 1 method)

## Second helper function

The second function is `update_cEGM!(solution, economy)`, where:
* `solution::AiyagariSolution` is a mutable `struct` `AiyagariSolution` which is updated by the function;
* `economy::Economy` is a immutable `struct` `Economy` which contains economy parameters.

The function updates the consumption policy function `gc` (in `solution`) using:
* previous period savings `agrid` (in `economy`), 
* current savings policy function `ga` (in `solution`), 
* model parameters in `economy`.    

In [2]:
function update_cEGM!(solution::AiyagariSolution,
                      economy::Economy)::Nothing
    
    @unpack Tt, u′,na,a_min,aGrid,ny,ys = economy
    @unpack ga,R,w,L = solution
    cs = similar(ga)

    for iy = 1:ny
        for ia = 1:na
            cs[ia,iy] = find_zero(c -> c-(R*aGrid[ia] + w*ys[iy]*L              
                -interpLinear(aGrid[ia], ga[:,iy], aGrid, na, a_min)[1] + Tt),
                one(w))
            # we solve for the agent's budget constraint (a fsolve is needed because
            # of the labor supply Euler equation). We use a linear interpolation for the
            # saving policy function -- which is somewhat 'inverted' because of the EGM
            # solution.
        end
    end
    solution.gc = cs
    return nothing
end;

## Third helper function


The third function is `EulerBackEGM!(solution, economy)`, where (as before):
* `solution::AiyagariSolution` is a mutable `struct` `AiyagariSolution` which is updated by the function;
* `economy::Economy` is a immutable `struct` `Economy` which contains economy parameters.

The function updates the policy functions for savings `ga`, for consumption `gc`, and for labor supply `gl` (all of them in `solution`) using the Euler equations for labor and consumption.

In [3]:
function EulerBackEGM!(solution::AiyagariSolution,
                       economy::Economy)::Nothing
    
    @unpack β,Tt,u′,inv_u′,na,aGrid,ny,ys,Πy = economy
    update_cEGM!(solution, economy) 
    @unpack gc,R,w,L = solution
    
    u′s_next = similar(gc)
    u′s_next .= u′.(gc)
    Eu′s = similar(gc)
    Eu′s .= β*R*u′s_next*Πy'
    
    cs = similar(gc) 
    as = similar(gc)
    
    cs .= inv_u′.(Eu′s)
    for ia = 1:na
        for iy = 1:ny
            as[ia,iy] = (aGrid[ia] + cs[ia,iy] - Tt - w*ys[iy]*L)/R                 
        end
    end
    # as is a(a')
    # cs is c(a')
    
    # updates policy function in solution
    solution.gc .= cs
    solution.ga .= as
    return nothing
end;

## Final function computing policy functions


The final function (the one actually computing policy functions) is `SolveEGM!(solution, economy; tol::Float64=1e-6, maxiter::Int64=10000)`, where (as before):
* `solution::AiyagariSolution` is a mutable `struct` `AiyagariSolution` which is updated by the function;
* `economy::Economy` is a immutable `struct` `Economy` which contains economy parameters;
* `tol::Float64` is a precision criterion to stop the convergence process;
* `maxiter::Int64` is a number of maximal repetitions (in case of non-convergence of policy function). The me

The function computes the policy functions for savings `ga`, for consumption `gc`, and for labor supply `gl` (all of them in `solution`) by iterating over the function `EulerBackEGM!`. 

The function stops when:
* either the difference between the policy function `ga` and its update is below the threshold `tol`;
* or the number of iterations is above the number of repetitions `maxiter`.
The output message is different in both cases.

In [4]:
function SolveEGM!(solution::AiyagariSolution,
                   economy::Economy;
                   tol::T=1e-6, maxiter::I=10000)::Nothing where {T<:Real,I<:Integer}
    # ierates on policy functions until convergence
    
    as  = similar(solution.ga)
    as .= solution.ga
    i = 0
    while true 
        i += 1
        EulerBackEGM!(solution, economy) #updates policy functions once
        if i%50 == 0
            # The convergence is only checked by blocks of 50 iterations            
            test = maximum(abs.(solution.ga .- as) ./ (
                abs.(solution.ga) .+ abs.(as))) #computation of the convergence criterion
            
            if test < tol
                # convergence is reached
                println("Solved in ",i," ","iterations");flush(stdout)
                break
            elseif i > maxiter
                # maximal nb of iterations is reached (but no convergence)
                println("Convergence not reached after ",i,"iterations. Criterion = ", test);flush(stdout)
                break
            else
                println("iteration: ", i , " ", maximum(test));flush(stdout)
                flush(stdout)
            end
        end
        as .= solution.ga
    end 
    return nothing
end;

# Computing stationary distribution

## The helper function

There is one helper function `transitionMat(ga,economy)` where:
* `ga::Matrix{T}` is the saving policy function (a `na`$\times$ `ny` matrix defined on the product grid of assets $\times$ productivity);

* `economy::Param` is the collection of model parameters.
NB: For types, we have: `T<:Real` and `I<:Integer`.

The function returns a sparse matrix (of type `SparseMatrixCSC{T,I}` from the package `SparseArrays`) such that:
* the sparse matrix is of size `na*ny`$\times$`na*ny`;
* a matrix element corresponds to the probability to switch from a pair of (savings, productivity level) to another pair of (savings, productivity level).

In [5]:
function transitionMat(ga::Matrix{T}, economy::Economy{T,T1,T2,T3,T4,T5,T6,T7,T8,I})::SparseMatrixCSC{T,I} where {
                            T<:Real,T1<:Function,T2<:Function,T3<:Function,T4<:Function,
                            T5<:Function,T6<:Function,T7<:Function,T8<:Function,
                            I<:Int64}
    @unpack na,a_min,aGrid,ny,Πy = economy
    
    trans  = spzeros(T, I, na*ny, na*ny)
    p      = zero(T)
    a_val  = zero(T)
    ia_val = zero(I)
    i_mat  = zero(I)
    j_mat  = zero(I)
    for ia = 1:na
        for iy = 1:ny
            a_val, ia_val = interpLinear(aGrid[ia], ga[:,iy], aGrid, na, a_min) 
            p = (aGrid[ia] - ga[ia_val,iy])/(
                    ga[ia_val+1,iy] - ga[ia_val,iy])
            p = min(max(p,zero(T)),one(T))
            #i_mat = (iy-1)*na
            j_mat = (iy-1)*na
            for jy = 1:ny
                #j_mat = (jy-1)*na
                i_mat = (jy-1)*na
                trans[i_mat+ia_val+1,j_mat+ia] = p * Πy[iy,jy]
                trans[i_mat+ia_val,j_mat+ia] = (one(T)-p) * Πy[iy,jy]
            end
        end
    end   
    return trans
end;

In [ ]:
# returns the index i such that: xs[i]≤x<xs[i+1]
# xs is assumed to be sorted in increasing order
# we impose bouds on i such that i≥1 and i+1≤length(xs)
function myfindbtwn(x,xs)
    i = findfirst(xs.≥x)
    if isnothing(i)
        return length(xs)-1
    elseif i==1
        return i
    else
        return i-one(typeof(i))
    end
end;

function transitionMat′(ga::Matrix{T}, economy::Economy{T,T1,T2,T3,T4,T5,T6,T7,T8,I})::SparseMatrixCSC{T,I} where {
                            T<:Real,T1<:Function,T2<:Function,T3<:Function,T4<:Function,
                            T5<:Function,T6<:Function,T7<:Function,T8<:Function,
                            I<:Int64}
    @unpack na,a_min,aGrid,ny,Πy = economy
    
    trans  = spzeros(T, I, na*ny, na*ny)
    p      = zero(T)
    a_val  = zero(T)
    ia_val = zero(I)
    i_mat  = zero(I)
    j_mat  = zero(I)
    for ia = 1:na
        for iy = 1:ny
            a_val   = ga[ia,iy]
            ia_val  = myfindbtwn(a_val,aGrid)
            p = (a_val - aGrid[ia_val])/(aGrid[ia_val+1] - aGrid[ia_val])
            p = min(max(p,zero(T)),one(T))
            i_mat = (iy-1)*na
            for jy = 1:ny
                j_mat = (jy-1)*na
                trans[i_mat+ia,j_mat+ia_val+1] = p * Πy[iy,jy]
                trans[i_mat+ia,j_mat+ia_val] = (one(T)-p) * Πy[iy,jy]
            end
        end
    end   
    return trans'
end;

## The final function

The function actually computing the stationary distribution is `stationaryDist(M; tol::Float64=1e-16, maxiter::Int64=100000)`, where:
* `M::SparseMatrixCSC{T,I}` is a (sparse) transition matrix that results from function `transitionMat`;
* `tol::Float64 = 1e-6` is a precision criterion to stop the convergence process;
* `maxiter::Int64=10000` is a number of maximal repetitions (in case of non-convergence of the computation).

 The function returns a vector $\Pi$ of size `na*ny` that verifies $\Pi M=\Pi$ and is  stationary distribution -- that is known to exist. It is computed as the normalised eigenvector corresponding to the largest eigen value of the transition matrix -- which is $1$. To do so, we rely on the function `powm!` of the package `IterativeSolvers`.

In [6]:
function statDist(M::SparseMatrixCSC{T,I}; tol::T=1e-16, maxiter::I=100000
        )::Vector{T} where {T<:Real,I<:Integer}
    nM = size(M)[1]
    _, x = powm!(Matrix(M), ones(T, nM), 
                    maxiter = maxiter,tol = tol)
    # returns the approximate largest eigenvalue λ of M and one of its eigenvector
    return x/sum(x)
end;

# Computing the steady-state equilibrium

## The main function

The function `steady(economy; tol::T=1e-8, maxiter::I=100000)` computes the steady-state solution of the Aiyigary model, where (as before):
* `economy::Economy` is a immutable `struct` `Economy` which contains economy parameters;
* `tol::T=1e-8` is a precision criterion to stop the convergence process;
* `maxiter::I=100000` is a number of maximal repetitions (in case of non-convergence of computations). The me

The function returns the steady-state allocation under the form of a mutable `struct` of type `AiyagariSolution{T,I}`. 

The function `steady`relies on previous functions, in particular `SolveEGM!` for computing steady-state policy functions and `stationaryDist` to compute the stationnary distribution.

In [8]:
function steady(economy::Economy;
                tolEGM::T=1e-8, maxiterEGM::I=100000, 
                tolSD::T=1e-16, maxiterSD::I=500000,
                initL::T=0.5,tolL::T=1e-6, maxiterL::I=100)::AiyagariSolution{T,I} where {T<:Real,I<:Integer}

    @unpack β,α,δ,τl,Tt,τk,u′,inv_v′,v′,na,a_min,aGrid,ny,ys,ϵw,κw = economy
    solution = AiyagariSolution(economy)
    @unpack ga,gc,L,w = solution
    # we iterate on L until v'(L) = (ϵw -1)/ϵw wₜ∫ᵢyᵢₜu′(cᵢₜ)ℓ(di)
    
    iterL = 0
    L0    = initL
    old_L = L0 
    relax = 0.5
    as = similar(ga)  #policy rules as a function of beginning of period asset
    cs = similar(ga)  #policy rules as a function of beginning of period asset
    transitMat = spzeros(typeof(β), typeof(na), na*ny, na*ny)
    stationaryDist = zeros(typeof(β), na, ny)

    


    while true
        solution.L = L0        
        SolveEGM!(solution, economy, tol=tolEGM, maxiter=maxiterEGM)
        @unpack ga,R,w,L = solution
        transitMat = transitionMat(ga,economy)
        stationaryDist = reshape(statDist(transitMat,tol=tolSD,maxiter=maxiterSD), na, ny)
        for ia = 1:na
            for iy = 1:ny
                as[ia,iy] = interpLinear(aGrid[ia], ga[:,iy], aGrid, na, a_min)[1]
                cs[ia,iy] = R*aGrid[ia] - as[ia,iy] + w*ys[iy]*L + Tt
            end
        end
        # we can check that norm(transitionMat′(as,economy) .- transitMat) small
        

        old_L = L0
        
        L0    = inv_v′(κw*(1-1/ϵw)*w*sum(stationaryDist.*((repeat(ys,1,na)'.*u′.(cs))))) 
        L0    = relax*L0 + (1-relax)*old_L
        iterL += 1


        @show iterL,L0,abs(L0-old_L)
        if (iterL > maxiterL)||(abs(L0-old_L) < tolL)
            solution.ga = as
            solution.gc = cs
            solution.transitMat = transitMat
            solution.stationaryDist = stationaryDist
            break
        end     
    end
    


    
    solution.gl = fill(solution.L,size(cs))
    
    # We compute aggregate quantities
    solution.A = sum(solution.stationaryDist.*solution.ga) #aggregate savings
    solution.C = sum(solution.stationaryDist.*solution.gc) #aggregate consumption
    # solution.K = L*α/(1/β-1+δ)^(1/(1-α)) #aggregate capital
    solution.K = solution.L*((1/β-1+δ)/α)^(1/(α-1)) #aggregate capital
    solution.Y = (solution.K)^α*(solution.L)^(1-α)

    solution.B = solution.A-solution.K
    solution.G = solution.Y- δ*solution.K - solution.C 
    
    # We check Euler equations by computing their residuals
    solution.residEuler = u′.(solution.gc) - β*solution.R*reshape(
    solution.transitMat'*reshape(u′.(solution.gc),na*ny,1),na,ny)
    
    return solution
end;

LoadError: LoadError: UndefVarError: @unpack not defined
in expression starting at In[8]:5

## Checking the consistency of the solution

In [9]:
function check_solution(solution::AiyagariSolution, economy::Economy)::Nothing
    @unpack β,α,δ,τl,τs,τk,Tt,na,a_min,aGrid,ny,ys = economy
    @unpack ga,gc,R,w,A,K,C,L,G,stationaryDist = solution
    if !(sum(stationaryDist) ≈ 1.0)
        @warn("error in stationary distribution. Sum: ", round(sum(stationaryDist),digits=4))
    else
        println("Passed: stationary distribution. Sum: ", round(sum(stationaryDist),digits=4))
    end
    if !(L*α/(R/(1-τk)-1+δ)^(1/(1-α)) ≈ K)
        @warn("error in capital. K=", round(K,digits=4), "; L*(K_FB/L_FB)=", 
            round(L*((1/β - (1-δ))/α)^(1/(α-1)),digits=4))
    else
        println("Passed: capital. K=", round(K,digits=4), "; L*(K_FB/L_FB)=", 
            round(L*((1/β - (1-δ))/α)^(1/(α-1)),digits=4))
    end
    if !(A ≈ sum(stationaryDist.*ga))
        @warn("error in aggregate savings. A=",round(A,digits=4), 
            "; ∫aᵢℓ(di)=",round(sum(stationaryDist.*ga),digits=4))
    else
        println("Passed: aggregate savings. A=",round(A,digits=4), 
            "; ∫aᵢ′ℓ(di)=",round(sum(stationaryDist.*ga),digits=4))
    end
    if abs(A-sum(stationaryDist.*repeat(economy.aGrid,1,ny))) > na*ny*1e-10
        @warn("error in convergence for savings. A=",round(A,digits=4), 
            "; ∫aᵢℓ(di)=",round(sum(stationaryDist.*repeat(economy.aGrid,1,ny)),digits=4))
    else
        println("Passed: convergence for savings. A=",round(A,digits=4), 
            "; ∫aᵢℓ(di)=",round(sum(stationaryDist.*repeat(economy.aGrid,1,ny)),digits=4))
    end
    C_ = w*L - A + Tt + R*sum(stationaryDist.*repeat(economy.aGrid,1,ny))
    if !(C ≈ C_)
        @warn("error in aggregate consumption. C=",round(C,digits=4), 
            "; ∫cᵢ ℓ(di)=",round(C_,digits=4))
    else
        println("Passed: aggregate consumption. C=",round(C,digits=4), 
            "; ∫cᵢ ℓ(di)=",round(C_,digits=4))
    end
    gov_bc = K^α * L^(1-α) - δ*K - (R-1)*A - w*L - G - Tt
    if (abs(gov_bc) > 1e-10)
        @warn("error in govt budget constraint. Gap=",round(gov_bc,sigdigits=4))
    else
        println("Passed: govt budget constraint. Gap=",round(gov_bc,sigdigits=4))
    end
    rc = K^α * L^(1-α) - δ*K - C - G
    if (abs(rc) > 1e-10)
        @warn("error in resource constraint. Gap=",round(rc,sigdigits=4))
    else
        println("Passed: resource constraint. Gap=",round(rc,sigdigits=4))
    end
    
    # correcting for consumption tax
    τc = 0.0
    R_init = R
    w_init = w*(1+τc)
    A_init = A*(1+τc)
    B_init = A_init - K
    
    gov_bc_init = K^α * L^(1-α) - δ*K +τc*C - (R_init-1)*A_init - w_init*L - G - Tt
    if (abs(gov_bc_init) > 1e-10)
        @warn("error in govt budget constraint (post tax). Gap=",round(gov_bc_init,sigdigits=4))
    else
        println("Passed: govt budget constraint (post tax). Gap=",round(gov_bc_init,sigdigits=4))
    end
    return nothing
end

# Describing the model solution

In [2]:
function describe_solution(solution::AiyagariSolution, economy::Economy; calib="quarterly")    
    @unpack β,α,δ,τk,τl,τs,Tt,u′,na,a_min,aGrid,ny,ys = economy
    @unpack ga,gc,Rt,wt,R,w,A,C,K,L,G,Y,B,stationaryDist,residEuler = solution
    T = typeof(β)
    τc = 0.
    # Adjusting stocks to the calibration (quarterly or yearly)
    stock_adj = occursin('q', lowercase(calib)) ? 4one(typeof(β)) : one(typeof(β))
     
    # Computing post-consumption-tax quantities
    R_init = R
    w_init = w
    A_init = A
    B_init = A_init - K
    
    # Computing MPC
    diff_gc = gc[2:end,:] - gc[1:end-1,:]
    diff_ga = aGrid[2:end,:] - aGrid[1:end-1,:]
    mpc = sum(diff_gc.*stationaryDist[1:end-1,:]./diff_ga)
    
    # Computing total fiscal revenues
    capital_tax = τk*(Rt-1)*A_init
    labor_tax   = wt*L - w*L
    conso_tax   = τc*C
    tot_tax     = capital_tax+labor_tax+conso_tax
    
    return Dict{String,T}("01. Gini" => Gini(ga, stationaryDist), 
                "02. Debt-to-GDP, B/Y"  => B_init/(stock_adj*Y), 
                "03. Public spending-to-GDP, G/Y"  => G/Y, 
                "04. Aggregate consumption-to-GDP, C/Y"  => C/Y,
                "05. Capital-to-GDP, K/Y"  => K/(stock_adj*Y),
                "06. Investment-to-GDP, I/Y"  => δ*K/Y,
                "07. Transfers-to-GDP, Tt/Y" => Tt/Y,
                "08. Aggregate labor supply, L" => L,
                "09. Average MPC" => mpc,
                "10. Consumption tax-to-GDP" => conso_tax/Y,
                "11. Labor tax-to-GDP" => labor_tax/Y,
                "12. Capital tax-to-GDP" => capital_tax/Y,
                "13. Total tax-to-GDP" => tot_tax/Y,
                "14. Share of credit-constrained agents" => sum(stationaryDist[1,:]))
end



LoadError: LoadError: UndefVarError: @unpack not defined
in expression starting at In[2]:2

describe_simulation (generic function with 1 method)

The function 
> `print_dict(dict::Dict{String,T}; sep=':',digits::Int64=4)::Nothing`

prints the description (label$\,\times\,$value) in the alphabetical order of the labels, with `digits` digits for the rounding and where label and value are separeted by `sep`.    


In [ ]:
function print_dict(dict::Dict{String,T}; sep="",digits=4)::Nothing where {T<:Real}
    tuples = sort([(k,v) for (k,v) in dict], by = first)
    max_label_length = maximum(map(length∘first,tuples))
    for (k,v) in tuples
        k_ = lowercase(k)
        trail_space = repeat(' ',1+max_label_length-length(k))
        to_pct = (occursin("to-gdp",k_)||occursin("share",k_)) 
        if to_pct
            println(k,sep*trail_space,round(100*v,digits=digits-2),"%")
        else
            println(k,sep*trail_space,round(v,digits=digits))
        end
    end
    return nothing
end;

The function 
> `restricted_trans(iy::I,solution::AiyagariSolution) where I<:Int`

returns the transition matrix and the stationary distribution corresponding to `solution` but restricted to idiosyncratic state `iy`.

In other words, multiplying $n$ times the obtained transition matrix  to the obtained stationary distribution allows one to obtain the stationary distribution of agents that have been at least for $n$ consecutive periods in state $n$.

In [ ]:
function restricted_trans(iy::I, solution::AiyagariSolution) where I<:Int
    transM = copy(solution.transitMat)
    statD  = copy(solution.stationaryDist)
    (na,ny)= size(statD)
    iy′    = iy
    for iy in 1:ny
        (iy==iy′) && continue
        for ia in 1:na
            statD[ia,iy] = zero(eltype(statD))
        end
    end
    for iy in 1:ny, jy in 1:ny
        (iy==iy′) && (jy==iy′) && continue
        for ia in 1:na, ja in 1:na
            transM[(iy-1)*na+ia,(jy-1)*na+ja] = zero(eltype(statD))
        end
    end
    return transM, statD
end

The function 
> `long_term_value(iy::I,solution::AiyagariSolution; policy::Symbol=:ga, tol=1e-6, maxiter=100) where I<:Int`

returns the value of the variable `policy` (by default asset, `:ga`) for agents remaining in state `iy` for a long period.

For instance, `long_term_value(iy,solution, policy=:gc)` returns the consumption level of agents remaining for a long time in state `iy`. 

In [1]:
function long_term_value(iy::I, solution::AiyagariSolution; symbol::Symbol=:ga, tol=1e-6, maxiter=100) where I<:Int
    gx = getproperty(solution,symbol)[:]
    transM_iy,statD_iy = restricted_trans(iy,solution);
    transM_iy_n = transM_iy^2500
    statD_iy_n  = transM_iy_n * statD[:]
    toR  = sum(gx.*statD_iy_n)/sum(statD_iy_n)
    toR_ = toR
    diff = one(eltype(gx))
    iter  = 0
    while (diff > tol)&&(iter<maxiter)        
        statD_iy_n = transM_iy_n * statD_iy_n
        toR_ = toR
        toR  = sum(gx.*statD_iy_n)/sum(statD_iy_n)
        diff = abs(toR-toR_)
        iter+= 1
    end
    return toR
end

LoadError: UndefVarError: AiyagariSolution not defined

In [ ]:
function long_term_path(iy::I, n::I, x0::T, solution::AiyagariSolution, economy::Economy;
         symbol::Symbol=:ga, tol=1e-6, maxiter=100) where {I<:Int,T<:Real}
    @unpack aGrid, na, a_min = economy
    MS = solution.stationaryDist
    gx = getproperty(solution,symbol)
    gx_foo(x) = interpLinear(x,aGrid,gx[:,iy],na,a_min)[1]
    
    toR = zeros(T,n)

    toR[1] = x0
    for i in eachindex(toR[2:end])
        toR[i+1] = gx_foo(toR[i])
    end
    return toR
end